In [2]:
import sys
import os
import pandas as pd
import numpy as np
import scipy
import gget
import sklearn.preprocessing 
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import fcluster
from importlib import reload
import matplotlib.pyplot as plt
from matplotlib import patches
import seaborn as sns
import networkx as nx
from matplotlib_venn import venn2
from matplotlib_venn import venn3
from scipy import sparse
import xgi

import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)

source_path = os.path.abspath("../../source/")
sys.path.append(source_path)
import utils as ut
import plotting as plt2
import hypercore as hc
import matrix as matrix
import centrality as central

In [7]:
"load pangloadb"
fpath = "../../resources/mESC_pangloadb.csv"
pdf = pd.read_csv(fpath)
print(f"{pdf.shape=}")
print(f"{pdf['gene_name'].nunique()=}")
print()

""" load gene ontology annotations """
fpath = "../../resources/stem_cell_population_maintenance.csv"
godf = pd.read_csv(fpath)
godf['gene_name'] = godf['gene_name'].str.upper()
print(f"{godf.shape=}")
print(f"{godf['gene_name'].nunique()=}")
print()

""" load gene expression """
fpath = "/scratch/indikar_root/indikar1/shared_data/higher_order/rna/expression.parquet"
exdf = pd.read_parquet(fpath)

tpm_columns = [x for x in exdf.columns if "TPM" in x]
exdf['expression_mean'] = exdf[tpm_columns].mean(axis=1)
exdf['gene_name'] = exdf['gene_name'].str.upper()
express_map = dict(zip(exdf['gene_name'].values, exdf['expression_mean'].values))

exdf = exdf[exdf['expression_mean'] > 0]
print(f"{exdf.shape=}")
print(f"{exdf['gene_name'].nunique()=}")

""" Gene locations """


gene_table_path = "/scratch/indikar_root/indikar1/shared_data/higher_order/reference/gene_table.parquet"
gdf = pd.read_parquet(gene_table_path)

gdf['gene_name'] = gdf['gene_name'].str.upper()

gdf['is_gene'] = [True] * len(gdf)
gdf['is_pt_gene'] = gdf['gene_biotype'] == 'protein_coding'
gdf['is_expressed'] = gdf['gene_name'].isin(exdf['gene_name'].values)
gdf['expression'] = gdf['gene_name'].map(express_map)
gdf['expression'] = gdf['expression'].fillna(0)
gdf['expression_log'] = np.log1p(gdf['expression'])
gdf['mESC_panglaoDB_marker'] = gdf['gene_name'].isin(pdf['gene_name'].values)
gdf['mESC_GO_marker'] = gdf['gene_name'].isin(godf['gene_name'].values)

print(f"{gdf.shape=}")

gdf.head()

pdf.shape=(97, 5)
pdf['gene_name'].nunique()=93

godf.shape=(218, 2)
godf['gene_name'].nunique()=218

exdf.shape=(23244, 17)
exdf['gene_name'].nunique()=23225
gdf.shape=(56655, 17)


,gene_id,gene_name,gene_source,gene_biotype,Chromosome,Start,End,length,midpoint,is_tf,is_gene,is_pt_gene,is_expressed,expression,expression_log,mESC_panglaoDB_marker,mESC_GO_marker
0,ENSMUSG00000104478,GM38212,havana,TEC,1,108344806,108347562,2756,108346184,False,True,False,False,0.00,0.000000,False,False
1,ENSMUSG00000104385,GM7449,havana,processed_pseudogene,1,6980783,6981446,663,6981114,False,True,False,False,0.00,0.000000,False,False
2,ENSMUSG00000102135,GM37108,havana,processed_pseudogene,1,6986782,6993812,7030,6990297,False,True,False,True,0.55,0.438255,False,False
3,ENSMUSG00000103282,GM37275,havana,processed_pseudogene,1,6999982,7000012,30,6999997,False,True,False,False,0.00,0.000000,False,False
4,ENSMUSG00000101097,GM6679,havana,processed_pseudogene,1,108697864,108699733,1869,108698798,False,True,False,False,0.00,0.000000,False,False


In [8]:
outpath = "../../resources/genes.parquet"
gdf.to_parquet(outpath, index=False)
gdf.head()

,gene_id,gene_name,gene_source,gene_biotype,Chromosome,Start,End,length,midpoint,is_tf,is_gene,is_pt_gene,is_expressed,expression,expression_log,mESC_panglaoDB_marker,mESC_GO_marker
0,ENSMUSG00000104478,GM38212,havana,TEC,1,108344806,108347562,2756,108346184,False,True,False,False,0.00,0.000000,False,False
1,ENSMUSG00000104385,GM7449,havana,processed_pseudogene,1,6980783,6981446,663,6981114,False,True,False,False,0.00,0.000000,False,False
2,ENSMUSG00000102135,GM37108,havana,processed_pseudogene,1,6986782,6993812,7030,6990297,False,True,False,True,0.55,0.438255,False,False
3,ENSMUSG00000103282,GM37275,havana,processed_pseudogene,1,6999982,7000012,30,6999997,False,True,False,False,0.00,0.000000,False,False
4,ENSMUSG00000101097,GM6679,havana,processed_pseudogene,1,108697864,108699733,1869,108698798,False,True,False,False,0.00,0.000000,False,False
